# Memento (Korean)

    객체의 변경 내역을 기록하여 복원 기능을 구현

## 정의

객체의 이전 상태를 저장하고 롤백할 수 있는 패턴.

텍스트 편집기의 ctrl + z의 동작 방식, 바둑 시뮬레이터의 복기 등을 예로 들 수 있을 것 같다.

## 구현

In [46]:
from datetime import datetime
from dataclasses import dataclass, field


class Originator:

    # 메멘토 객체는 불변해야 한다
    @dataclass(frozen=True)
    class Snapshot:
        title: str
        data: str
        create_time: datetime = field(default_factory=datetime.now)
    
    def __init__(self, title, data):
        self.title = title
        self.data = data
    
    # 메멘토 객체를 생성하고
    def create_snapshot(self):
        return self.Snapshot(self.title, self.data)
    
    # 메멘토 객체를 활용해 복원하고
    def restore(self, snapshot: Snapshot):
        self.title = snapshot.title
        self.data = snapshot.data
    
    def __repr__(self) -> str:
        return f"{type(self).__name__}({self.title}, {self.data})"


# 메멘토 객체들을 관리한다
class Caretaker:

    def __init__(self, originator: Originator):
        self.originator = originator
        self.history: list[Originator.Snapshot] = []
    
    def backup(self):
        self.history.append(self.originator.create_snapshot())
    
    def undo(self):
        if not self.history:
            return
        snapshot = self.history.pop()
        self.originator.restore(snapshot)
    
    def show_history(self):
        for snapshot in self.history:
            print(snapshot)


originator = Originator("제목", "본문")
print(originator, end="\n\n")

caretaker = Caretaker(originator)
caretaker.backup()
originator.title = "다른 제목"
caretaker.backup()
originator.data = "다른 본문"
caretaker.show_history()
print(originator, end="\n\n")

caretaker.undo()
caretaker.undo()
print(caretaker.originator)

Originator(제목, 본문)

Originator.Snapshot(title='제목', data='본문', create_time=datetime.datetime(2021, 12, 30, 20, 11, 20, 999332))
Originator.Snapshot(title='다른 제목', data='본문', create_time=datetime.datetime(2021, 12, 30, 20, 11, 20, 999332))
Originator(다른 제목, 다른 본문)

Originator(제목, 본문)


In [51]:
# 커맨드 패턴과 함께 활용
from datetime import datetime
from dataclasses import dataclass, field


class Originator:

    # 메멘토 객체를 커맨드 객체로 활용
    # (요청에 대한 모든 정보가 포함된 독립 실행형 객체)
    @dataclass(frozen=True)
    class Snapshot:
        originator: Originator
        title: str
        data: str
        create_time: datetime = field(default_factory=datetime.now)

        # !
        def restore(self):
            self.originator.title = self.title
            self.originator.data = self.data
    
    def __init__(self, title, data):
        self.title = title
        self.data = data
    
    def create_snapshot(self):
        # !
        return self.Snapshot(self, self.title, self.data)
    
    def __repr__(self) -> str:
        return f"{type(self).__name__}({self.title}, {self.data})"


class Caretaker:

    def __init__(self, originator: Originator):
        self.originator = originator
        self.history: list[Originator.Snapshot] = []
    
    def backup(self):
        self.history.append(self.originator.create_snapshot())
    
    def undo(self):
        if not self.history:
            return
        snapshot = self.history.pop()
        # !
        snapshot.restore()
    
    def show_history(self):
        for snapshot in self.history:
            print(f"{snapshot.create_time} / {snapshot.title} - {snapshot.data}")


originator = Originator("제목", "본문")
print(originator, end="\n\n")

caretaker = Caretaker(originator)
caretaker.backup()
originator.title = "다른 제목"
caretaker.backup()
originator.data = "다른 본문"
caretaker.show_history()
print(originator, end="\n\n")

caretaker.undo()
caretaker.undo()
print(caretaker.originator)

Originator(제목, 본문)

2021-12-30 20:20:38.590651 / 제목 - 본문
2021-12-30 20:20:38.590651 / 다른 제목 - 본문
Originator(다른 제목, 다른 본문)

Originator(제목, 본문)
